<a href="https://colab.research.google.com/github/WaliSiddiqui1/BPM/blob/main/NSVA_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

!pip install yt-dlp webvtt-py
!pip install opencv-python-headless

import os
import cv2
import json
import time
import random
import numpy as np
from pathlib import Path
from yt_dlp import YoutubeDL
import webvtt
from google.colab import files

DRIVE_PATH = "/content/drive/MyDrive/NSVA_Data"
VIDEO_DIR = os.path.join(DRIVE_PATH, "videos")
CLIP_DIR = os.path.join(DRIVE_PATH, "data/clips")
CAPTION_DIR = os.path.join(DRIVE_PATH, "data/captions")
FRAME_DIR = os.path.join(DRIVE_PATH, "data/frames")

os.makedirs(VIDEO_DIR, exist_ok=True)
os.makedirs(CLIP_DIR, exist_ok=True)
os.makedirs(CAPTION_DIR, exist_ok=True)
os.makedirs(FRAME_DIR, exist_ok=True)

print("Please upload your youtube_cookies.txt file:")
uploaded_cookies = files.upload()
COOKIES_FILE = list(uploaded_cookies.keys())[0]

print("Please upload your video_links.txt file:")
uploaded_links = files.upload()
VIDEO_LINKS_FILE = list(uploaded_links.keys())[0]

try:
    gpu_mat = cv2.cuda_GpuMat()
    has_cuda = True
    print("CUDA is available for OpenCV acceleration")
except:
    has_cuda = False
    print("CUDA is not available for OpenCV, using CPU mode")

def download_video_and_captions(youtube_url):
    if not os.path.exists(COOKIES_FILE):
        print(f"Warning: Cookie file {COOKIES_FILE} not found.")
        print("You may encounter authentication issues.")

    ydl_opts = {
        'format': 'bestvideo+bestaudio',
        'outtmpl': os.path.join(VIDEO_DIR, '%(title)s.%(ext)s'),
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en'],
        'skip_download': False,
        'cookiefile': COOKIES_FILE if os.path.exists(COOKIES_FILE) else None,
        'quiet': False,
        'no_warnings': False
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])

        delay = random.uniform(7, 15)
        print(f"Waiting {delay:.2f} seconds before next operation...")
        time.sleep(delay)
        return True
    except Exception as e:
        print(f"Error downloading {youtube_url}: {e}")
        return False

def segment_video(video_path, clip_duration=5):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    length_in_sec = total_frames // fps
    video_name = Path(video_path).stem
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    clip_paths = []
    for i in range(0, length_in_sec, clip_duration):
        cap.set(cv2.CAP_PROP_POS_MSEC, i * 1000)
        out_path = os.path.join(CLIP_DIR, f"{video_name}_clip_{i}.mp4")

        if has_cuda:
            out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
            for _ in range(clip_duration * fps):
                ret, cpu_frame = cap.read()
                if not ret:
                    break

                gpu_frame = cv2.cuda_GpuMat()
                gpu_frame.upload(cpu_frame)

                out.write(gpu_frame.download())
        else:
            out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
            for _ in range(clip_duration * fps):
                ret, frame = cap.read()
                if not ret:
                    break
                out.write(frame)

        out.release()
        clip_paths.append((out_path, i))

    cap.release()
    return clip_paths

def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = [int(x) for x in list(
        map(lambda x: x * total_frames // num_frames, range(num_frames))
    )]

    frames = []
    for i in idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            if has_cuda:
                gpu_frame = cv2.cuda_GpuMat()
                gpu_frame.upload(frame)
                gpu_resized = cv2.cuda.resize(gpu_frame, (224, 224))
                resized_frame = gpu_resized.download()
                frames.append(resized_frame)
            else:
                resized_frame = cv2.resize(frame, (224, 224))
                frames.append(resized_frame)

    cap.release()
    return frames

def parse_captions(vtt_path):
    captions = []
    for caption in webvtt.read(vtt_path):
        if caption.text.strip():
            start = caption.start
            end = caption.end
            text = caption.text.strip().replace('\n', ' ')
            captions.append((start, end, text))
    return captions

def timestamp_to_seconds(ts):
    h, m, s = ts.split(':')
    s, ms = s.split('.')
    return int(h)*3600 + int(m)*60 + int(s)

def align_captions_to_clips(captions, clip_start, clip_end):
    result = []
    for start, end, text in captions:
        start_s = timestamp_to_seconds(start)
        end_s = timestamp_to_seconds(end)
        if clip_start <= start_s < clip_end:
            result.append(text)
    return " ".join(result)

def process_video(video_path, vtt_path):
    video_name = Path(video_path).stem
    captions = parse_captions(vtt_path)
    clips = segment_video(video_path)

    batch_size = 4
    for i in range(0, len(clips), batch_size):
        batch_clips = clips[i:i+batch_size]

        for clip_path, start_time in batch_clips:
            end_time = start_time + 5
            aligned_caption = align_captions_to_clips(captions, start_time, end_time)

            if not aligned_caption.strip():
                continue

            frames = extract_frames(clip_path)
            frames_out_path = os.path.join(FRAME_DIR, f"{Path(clip_path).stem}.json")
            frame_data = [frame.tolist() for frame in frames]

            data_entry = {
                "clip_id": Path(clip_path).stem,
                "frames": frame_data,
                "caption": aligned_caption,
                "metadata": {
                    "source_video": video_name,
                    "start_time": start_time,
                    "end_time": end_time
                }
            }

            with open(frames_out_path, 'w') as f:
                json.dump(data_entry, f)

def download_and_process_video_list(links_file, max_retries=3):
    print(f"Starting download and processing of videos listed in {links_file}")

    if not os.path.exists(links_file):
        print(f"Error: Video links file {links_file} not found!")
        return

    with open(links_file, 'r') as f:
        links = [line.strip() for line in f if line.strip()]

    print(f"Found {len(links)} video links in {links_file}")

    for i, video_url in enumerate(links):
        print(f"Processing video {i+1}/{len(links)}: {video_url}")
        for attempt in range(max_retries):
            try:
                success = download_video_and_captions(video_url)
                if success:
                    break
                print(f"Download failed on attempt {attempt+1}/{max_retries}, retrying...")
                time.sleep(random.uniform(10, 20))
            except Exception as e:
                print(f"Error on attempt {attempt+1}: {e}")
                if attempt < max_retries - 1:
                    print("Retrying...")
                    time.sleep(random.uniform(15, 30))
                else:
                    print(f"Failed to download {video_url} after {max_retries} attempts, skipping.")

if __name__ == "__main__":
    if not os.path.exists(COOKIES_FILE):
        print("=" * 80)
        print(f"WARNING: Cookie file {COOKIES_FILE} not found!")
        print("You may encounter 'Sign in to confirm you're not a bot' errors.")
        print("=" * 80)
    else:
        print(f"Found cookies file: {COOKIES_FILE}")
    download_and_process_video_list(VIDEO_LINKS_FILE)

    for file in os.listdir(VIDEO_DIR):
        if file.endswith(".mp4") or file.endswith(".webm"):
            base_name = Path(file).stem
            vtt_file = os.path.join(VIDEO_DIR, base_name + ".en.vtt")
            video_file = os.path.join(VIDEO_DIR, file)
            if os.path.exists(vtt_file):
                print(f"Processing video file: {file}")
                process_video(video_file, vtt_file)
            else:
                print(f"Warning: No caption file found for {file}")

/bin/bash: line 1: nvidia-smi: command not found
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 85.5 MB/s eta 0:00:00
Please upload your youtube_cookies.txt file:


Saving youtube_cookies.txt to youtube_cookies.txt
Please upload your video_links.txt file:


Saving video_links.txt to video_links.txt
CUDA is available for OpenCV acceleration
Found cookies file: youtube_cookies.txt
Starting download and processing of videos listed in video_links.txt
Found 1024 video links in video_links.txt
Processing video 1/1024: https://www.youtube.com/watch?v=AaWrXz9Y5HE
[youtube] Extracting URL: https://www.youtube.com/watch?v=AaWrXz9Y5HE
[youtube] AaWrXz9Y5HE: Downloading webpage
[youtube] AaWrXz9Y5HE: Downloading tv client config
[youtube] AaWrXz9Y5HE: Downloading player 22f02d3d-main
[youtube] AaWrXz9Y5HE: Downloading tv player API JSON
[info] AaWrXz9Y5HE: Downloading subtitles: en
[info] AaWrXz9Y5HE: Downloading 1 format(s): 298+251
[info] Writing video subtitles to: /content/drive/MyDrive/NSVA_Data/videos/LA Clippers vs Indiana Pacers Full Game Highlights ｜ March 19, 2018-19 NBA Season.en.vtt
[download] Destination: /content/drive/MyDrive/NSVA_Data/videos/LA Clippers vs Indiana Pacers Full Game Highlights ｜ March 19, 2018-19 NBA Season.en.vtt
[down

KeyboardInterrupt: 